In [43]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import streamlit as st

In [17]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [1]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [45]:
documents

[Document(id_='3cfef715-100b-41ce-900d-556856e6bd8b', embedding=None, metadata={'page_label': '1', 'file_name': 'crimes.pdf', 'file_path': 'c:\\IRR\\data\\crimes.pdf', 'file_type': 'application/pdf', 'file_size': 244628, 'creation_date': '2024-04-26', 'last_modified_date': '2024-04-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="crimes\nPage 1Title\n0Day-care center owner gets 4 years for shooting her husband\n1New video shows Rand Paul staffer getting stabbed repeatedly\n2One critically wounded after U.S. Marshals task force opens fire\n3While you were sleeping, the acting D.C. police chief could not\n4Water utility files suit over chemicals\n5Local Digest\n6Coalition seeks overhaul to D.C.'s crisis response\n7Ex-FBI supervisor arrested in Jan. 6 

In [2]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

c:\IRR\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 156/156 [00:01<00:00, 87.07it/s]


In [3]:
query_engine=index.as_query_engine()

In [4]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocessor])



In [5]:
response=query_engine.query("when were two people fatally shot")

In [41]:

print(response)

The newly released footage shows for the first time the brazen March 25 attack.


In [6]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: Two people were fatally shot on Thursday morning.
______________________________________________________________________
Source Node 1/4
Node ID: 37db78db-961a-4dc1-8e3f-0157dfde7623
Similarity: 0.8364014525682242
Text: Metro 2 killed near Nationals Park lived close to scene  Peter
Hermann  459 words 25 December 2023 The Washington Post WP FINAL B03
English Copyright 2023, The Washington Post Co. All Rights Reserved
D.C. police have identified two people fatally shot Thursday morning
near Nationals Park in Southwest Washington as a young man and a
35-year-old...
______________________________________________________________________
Source Node 2/4
Node ID: 68c78b36-1b22-4cdf-be8a-a35c66f894be
Similarity: 0.8156608733862033
Text: crimes Page 87A 15-year-old girl, who D The crime left
Wilkinson's family reel The D.C. Court of Appeals on Leonard was shot
13 to 15 times, according to The shooting has sparked a fiery The
family of Lazarus David Wilson, who A Philadelphia man

In [1]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.response.pprint_utils import pprint_response

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("safety in fourth of july in downtown dc parade")
print(response)
pprint_response(response,show_source=True)

Safety during the Fourth of July in downtown DC parade can be a concern due to the historical violence that has occurred in the city during this holiday. The city has faced challenges in controlling crime, with incidents of shootings and fatalities happening during Independence Day celebrations. The police have struggled to manage the violence, and the city has seen an increase in homicides, making it one of the deadliest years in two decades. Efforts have been made to deploy non-law enforcement officials, like violence interrupters, to de-escalate tensions in high-risk areas during the holiday period.
Final Response: Safety during the Fourth of July in downtown DC parade
can be a concern due to the historical violence that has occurred in
the city during this holiday. The city has faced challenges in
controlling crime, with incidents of shootings and fatalities
happening during Independence Day celebrations. The police have
struggled to manage the violence, and the city has seen an in

In [2]:
import streamlit as st
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.response.pprint_utils import pprint_response

# Page configuration
st.set_page_config(page_title="Crime Analysis", layout="centered", initial_sidebar_state="collapsed")

# Initialize or load index
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
query_engine = index.as_query_engine()

# Streamlit interface
st.title("Crime Analysis")
query = st.text_input("Enter the time and place you want to analyze", "")

col1, col2 = st.columns(2)
with col1:
    response_length = st.number_input("How many words in the response?", min_value=50, max_value=1000, value=200, step=50)
with col2:
    response_purpose = st.selectbox(
        "Generating response for:",
        options=["Research", "Educational", "Safety"]
    )

if st.button("Generate"):
    if query:
        response = query_engine.query(query)
        st.write("Response:")
        st.text(pprint_response(response, show_source=True))
    else:
        st.write("Please enter a query to analyze.")


2024-04-28 20:43:42.069 
  command:

    streamlit run c:\IRR\.conda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
